In [1]:
#Making necesarry imports
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
import ipywidgets as widgets
from IPython.display import display, clear_output
from contextlib import contextmanager
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os, sys
import re
import seaborn as sns

In [2]:
goods = pd.read_csv('./qf_goods.csv', header=None)
goods.columns = ['id', 'is_delete', 'title', 'price', 'clicks', 'collections', 'praises', 'bads', 'state', 'tag_id', 'sales']
goods.set_index('id')

,is_delete,title,price,clicks,collections,praises,bads,state,tag_id,sales
id,,,,,,,,,,
1,1,数码相机,499,68,10,5,1,1,1,10
2,1,单反,1400,103,50,10,1,1,1,5
3,1,显示器,999,89,30,10,20,1,1,50
4,1,耳机耳麦,2999,300,200,89,5,1,1,99
5,1,智能手表,499,500,200,61,0,1,1,99
...,...,...,...,...,...,...,...,...,...,...
224,0,抖音创意简装生活卫生间居家居日用品用具小百货家用大全清洁神器,3,6563,1849,3584,256,1,14,3996
225,0,家居家用小东西家庭厨房卫生间用品用具小百货大全居家生活日用品,5,3883,588,1262,852,1,14,2007
226,0,创意家居生活用品实用小百货大全居家用具厨房用品物件宿舍神器,16,1629,218,409,110,1,14,994


In [4]:
goods=goods.drop(['title'],axis=1)
goods

,id,is_delete,price,clicks,collections,praises,bads,state,tag_id,sales
0,1,1,499,68,10,5,1,1,1,10
1,2,1,1400,103,50,10,1,1,1,5
2,3,1,999,89,30,10,20,1,1,50
3,4,1,2999,300,200,89,5,1,1,99
4,5,1,499,500,200,61,0,1,1,99
...,...,...,...,...,...,...,...,...,...,...
223,224,0,3,6563,1849,3584,256,1,14,3996
224,225,0,5,3883,588,1262,852,1,14,2007
225,226,0,16,1629,218,409,110,1,14,994
226,227,0,12,1101,353,626,218,1,14,1043


In [6]:
# 删除 商品不在出售的
goods.drop(goods[goods.is_delete == True].index, inplace=True)
goods.drop(goods[goods.state != 1].index, inplace=True)
goods

,id,is_delete,price,clicks,collections,praises,bads,state,tag_id,sales
31,32,0,938,7327,2586,3429,919,1,1,5025
32,33,0,2399,5544,1790,2250,1224,1,1,3001
33,34,0,668,2751,619,1067,261,1,1,1991
34,35,0,4999,622,283,316,222,1,1,483
35,36,0,5699,112,33,61,7,1,1,71
...,...,...,...,...,...,...,...,...,...,...
223,224,0,3,6563,1849,3584,256,1,14,3996
224,225,0,5,3883,588,1262,852,1,14,2007
225,226,0,16,1629,218,409,110,1,14,994
226,227,0,12,1101,353,626,218,1,14,1043


In [8]:
goods['price_cut'] = pd.qcut(goods.price, q=10, labels=[i for i in range(10)])
goods

,id,is_delete,price,clicks,collections,praises,bads,state,tag_id,sales,price_cut
31,32,0,938,7327,2586,3429,919,1,1,5025,7
32,33,0,2399,5544,1790,2250,1224,1,1,3001,8
33,34,0,668,2751,619,1067,261,1,1,1991,6
34,35,0,4999,622,283,316,222,1,1,483,8
35,36,0,5699,112,33,61,7,1,1,71,9
...,...,...,...,...,...,...,...,...,...,...,...
223,224,0,3,6563,1849,3584,256,1,14,3996,0
224,225,0,5,3883,588,1262,852,1,14,2007,0
225,226,0,16,1629,218,409,110,1,14,994,1
226,227,0,12,1101,353,626,218,1,14,1043,0


In [10]:
pd.cut(goods.clicks, bins=[0, 100, 300, 500, 800, 1000, 2000, 4000, 6000, 10000, 100000], labels=[i for i in range(10)])
goods['clicks_cut'] = pd.cut(goods.clicks, bins=[0, 100, 300, 500, 800, 1000, 2000, 4000, 6000, 10000, 100000], labels=[i for i in range(10)])

In [12]:
goods['collections_cut'] = pd.qcut(goods.collections, q=10, labels=[i for i in range(10)])
goods['praises_cut'] = pd.qcut(goods.praises, q=10, labels=[i for i in range(10)])
goods['bads_cut'] = pd.qcut(goods.bads, q=10, labels=[i for i in range(10)])
goods['sales_cut'] = pd.qcut(goods.sales, q=10, labels=[i for i in range(10)])
goods

,id,is_delete,price,clicks,collections,praises,bads,state,tag_id,sales,price_cut,clicks_cut,collections_cut
31,32,0,938,7327,2586,3429,919,1,1,5025,7,8,8
32,33,0,2399,5544,1790,2250,1224,1,1,3001,8,7,7
33,34,0,668,2751,619,1067,261,1,1,1991,6,6,4
34,35,0,4999,622,283,316,222,1,1,483,8,3,2
35,36,0,5699,112,33,61,7,1,1,71,9,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,224,0,3,6563,1849,3584,256,1,14,3996,0,8,7
224,225,0,5,3883,588,1262,852,1,14,2007,0,6,4
225,226,0,16,1629,218,409,110,1,14,994,1,5,2
226,227,0,12,1101,353,626,218,1,14,1043,0,5,3


In [18]:

goods

,id,is_delete,price,clicks,collections,praises,bads,state,tag_id,sales,price_cut,clicks_cut,collections_cut,praises_cut,bads_cut,sales_cut
31,32,0,938,7327,2586,3429,919,1,1,5025,7,8,8,8,7,7
32,33,0,2399,5544,1790,2250,1224,1,1,3001,8,7,7,6,7,6
33,34,0,668,2751,619,1067,261,1,1,1991,6,6,4,5,4,5
34,35,0,4999,622,283,316,222,1,1,483,8,3,2,2,4,2
35,36,0,5699,112,33,61,7,1,1,71,9,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,224,0,3,6563,1849,3584,256,1,14,3996,0,8,7,8,4,7
224,225,0,5,3883,588,1262,852,1,14,2007,0,6,4,5,6,5
225,226,0,16,1629,218,409,110,1,14,994,1,5,2,2,2,4
226,227,0,12,1101,353,626,218,1,14,1043,0,5,3,3,4,4


In [20]:
d_goods = goods[['id', 'price_cut', 'clicks_cut', 'collections_cut', 'praises_cut', 'bads_cut', 'sales_cut']]
d_goods

,id,price_cut,clicks_cut,collections_cut,praises_cut,bads_cut,sales_cut
31,32,7,8,8,8,7,7
32,33,8,7,7,6,7,6
33,34,6,6,4,5,4,5
34,35,8,3,2,2,4,2
35,36,9,1,0,0,0,0
...,...,...,...,...,...,...,...
223,224,0,8,7,8,4,7
224,225,0,6,4,5,6,5
225,226,1,5,2,2,2,4
226,227,0,5,3,3,4,4


In [23]:
d_goods['score'] = d_goods[['clicks_cut']]+d_goods[['collections_cut']]+d_goods[['praises_cut']]-d_goods[['bads_cut']]+d_goods[['sales_cut']]
d_goods

TypeError: Object with dtype category cannot perform the numpy op add